<a href="https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/homeworks/language-model/hw4_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4 - Finetune GPT-2
This colab demonstrates how to fine-tune GPT-2 on a dataset of presidential speeches. We use the [Hugging Face Transformer](https://github.com/huggingface/transformers) library in order to do this.

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

## Setup

### Install HuggingFace Transfomers library.

In [1]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples/language-modeling')
!pip install dict_to_obj


Cloning into 'transformers'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 26447 (delta 52), reused 66 (delta 32), pack-reused 26353
Receiving objects: 100% (26447/26447), 15.95 MiB | 9.47 MiB/s, done.
Resolving deltas: 100% (18428/18428), done.
Processing /content/transformers
     |████████████████████████████████| 3.8MB 2.7MB/s 
     |████████████████████████████████| 1.1MB 24.5MB/s 
     |████████████████████████████████| 890kB 31.5MB/s 
  Created wheel for transformers: filename=transformers-2.9.1-cp36-none-any.whl size=637722 sha256=5ea344745dfcc3ea847f3ba80693a9daaeb3e93e2fa5f552833491b5156faedc
  Stored in directory: /tmp/pip-ephem-wheel-cache-p8n6ggjk/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=efdbcad8f334a09893e19f7637f37a7ae1c6caeba9bd25a5719de8e88f10d05b

In [0]:
import torch
import run_language_modeling
#import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

### Mount your Google Drive
We will be saving trained checkpoints on your Google Drive so that they can be accessed even if the Colab session dies. Make sure to login with your UPenn credentials, as you will be saving several gigabytes of data, and Penn gives you unlimited Drive storage.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Download data data.
It's already been split into a train, valid, and test set for you.

In [0]:
from requests import get
from bs4 import BeautifulSoup
import re

def parse_lyrics(song_link):
  response = get(song_link)
  html_soup = BeautifulSoup(response.text, 'html.parser')
  lyrics_containers = html_soup.find_all('div', class_ = 'lyrics')
  lyrics = html_soup.find_all('p')
  song = []
  for paragraph in lyrics:
    song.append(paragraph.get_text())
  del song[-2:]
  song = ' '.join(song)
  return song

In [20]:
url = 'https://www.nickcave.com/lyrics/'
response = get(url)

html_soup = BeautifulSoup(response.text, 'html.parser')
albums = html_soup.find_all('div', class_ = 'lyric-album-list')
print(f'Noumber of available albums: {len(albums)}')

Noumber of available albums: 20


In [21]:
file  = open("content/songs.txt", "w") 

no_songs = 0

for album in albums:
  songs = album.find_all('a')
  for song in songs:
    song_link = song.get('href')
    file.write(parse_lyrics(song_link)+'\n')
    no_songs += 1

print(f"No. of songs: {no_songs}")

No. of songs: 201


In [22]:
f = open("content/songs.txt", "r")
print(f.read())

Once there was a song The song yearned to be sung It was a spinning song About the king of rock ‘n’ roll The king was first a young prince The prince was the best With his black jelly hair He crashed onto a stage in Vegas The king had a queen The queen’s hair was a stairway She tended the castle garden And in the garden planted a tree The garden tree was a stairway It was sixteen branches high On the top branch was a nest Sing the high cloudy nest In the nest there was a bird The bird had a wing The wing had a feather Spin the feather and sing the wind The king in time died The queen’s heart broke like a vow And the tree returned to the earth With the nest and the bird But the feather spun upward Upward and upward Spinning all the weather vanes And you’re sitting at the kitchen table Listening to the radio And I love you And I love you And I love you And I love you And I love you And I love you And I love you Peace will come Peace will come Peace will come in time A time will come A ti

In [0]:
import random
lines = open('songs.txt').readlines()
random.shuffle(lines)

train_size = 0.9
train_songs = open('content/train_songs.txt', 'w').writelines(lines[:int(train_size*no_songs)])
valid_songs = open('content/valid_songs.txt', 'w').writelines(lines[int(train_size*no_songs):])

## Finetune and Eval
The Hugging Face library provides a script [run_language_modeling.py](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) which contains all of the code for training and evaluating a language model.

We will be calling this script directly from the command line in order to launch training. We will also use functions from this script to conduct evaluation and generate samples at inference time.

### Launch fine-tuninng
We will be calling `run_language_modeling.py` from the command line to launch fine-tuning, **Running fine-tuning may take several hours.** Every `save_steps` steps, a checkpoint is saved to disk. The checkpoint contains all the learned weights for your model, and you can  always reload the model from a saved checkpoint, even if your Colab has crashed.

Below is an explanation of some of the arguments you might want to modify in the command below. 

* `--line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `--num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `--block_size`: Your training text is truncated into blocks of this length. At test time, you will only want to generate sequences that are at most this length.
* `--gradient_accumulation_steps`: Update the model weights every this many steps. You shold set this to >1 when the batch size is very small to improve training stability.
* `--output_dir`: This is the where checkpoints will get saved. When you finetune on your own dataset, you should change this path. We recommend saving checkpoints to your Google Drive (`/content/drive/My Drive/`) so you can access them even if the Colab session dies.
* `--model_name_or_path` The path to the model weights to use when starting fine-tuning. You can set this to `gpt2-medium` to initialize with GPT-2's 355 million parameter model, or `gpt2` to initialize with their smaller 124 million parameter model. You can also set this to one of your own checkpoints to restart your training job if it crashes.

**I am getting out-of memory errors. What do I do?**

The number of trainable paramters in the model is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing these value.

**Oh no! My computer went to sleep and the Colab disconnected.**

The train job might have still completed. Check the `output_dir` in your Google Drive to see if checkpoint files have been created there.

**Training is taking foreverrrrrr.**

Try decreasing `num_train_epochs` or changing `model_name_or_path` to `gpt2` instead of `gpt2-medium`.
If your evaluation set is very large, you might also want to remove the `evaluate_during_training` flag or increase `logging_steps`.

In [0]:
# Get the example scripts.
!wget -c https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py

--2020-05-14 16:43:46--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-05-14 16:43:46 ERROR 404: Not Found.



In [0]:
!ls

__pycache__  README.md	run_language_modeling.py


In [4]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/songs' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/train_songs.txt \
    --do_eval \
    --eval_data_file=/content/valid_songs.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=64 \
    --gradient_accumulation_steps=5 #\
    #--overwrite_output_dir = True

2020-05-16 14:55:31.073780: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/16/2020 14:55:33 - INFO - transformers.training_args -   PyTorch: setting up devices
05/16/2020 14:55:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/16/2020 14:55:33 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/drive/My Drive/finetuned_models/songs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, per_gpu_train_batch_size=2, per_gpu_eval_batch_size=2, gradient_accumulation_steps=5, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=5, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [5]:
!ls '/content/drive/My Drive/finetuned_models/songs'

config.json	     pytorch_model.bin	      training_args.bin
eval_results_lm.txt  special_tokens_map.json  vocab.json
merges.txt	     tokenizer_config.json


#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  #tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)
  tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.eval_data_file(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [14]:
#import sys
#sys.path.append('../')
import importlib  
#run_language_modeling = importlib.import_module("language-modeling.run_language_modeling")
#os.chdir('/content/transformers/examples/language-modeling')
#from language-modeling.run_language_modeling import evaluate

# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/songs'
#CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/presidential_speeches' #'gpt2-medium'

# Set this to the list of text files you want to evaluate the perplexity of.
#DATA_PATHS = ["/content/valid_songs.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(#
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

!ls

05/16/2020 15:26:53 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/songs/config.json
05/16/2020 15:26:53 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

05/16/2020 15:26:53 - INFO - transform

Running on device:  cuda
pplm  __pycache__  README.md  run_generation.py


In [0]:
#for data_path in DATA_PATHS:
#  eval_results = do_perplexity_eval(args, model, data_path)
#  perplexity = eval_results['perplexity']
#  print('{} is the perplexity of {} according to {}'.format(
#      perplexity, data_path, CHECKPOINT_PATH))

### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  #tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)
  tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=None)

  os.chdir('/content/transformers/examples/text-generation')
  import run_generation
  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [18]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/songs'

# You should try out other prompts as well as no prompt at all.
PROMPT = '<title=\"Just like me\">'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=3,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

05/16/2020 15:44:26 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/songs/config.json
05/16/2020 15:44:26 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "vocab_size": 50257
}

05/16/2020 15:44:26 - INFO - transform

Running on device:  cuda


05/16/2020 15:44:42 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e05dc670366092d809a9cbcac6798e24e
05/16/2020 15:44:42 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-gene


====== GENERATION 0 ======
<title="Just like me">Just like me</title> I go up to you and say, you know she’s so niceYou go up to me and say, you know she’s so niceYou go up to me and say, you know she’s so niceYou go up to me and say, you know she ’s so niceYou go up to me and say, you know she’s that goodYou come up here and you know that she’s’

====== GENERATION 1 ======
<title="Just like me">A man and a crow A man and a rooster A man and a crowA man and a crowBut they did it to himA man on a horse A man on a horse A man on a black horse And a man on a black horse And a man on a black horse And a man on a black horse I knew you when I met you’d call me Jack’son, I could tell you’t be more than that a mere human, A mere mortal,

====== GENERATION 2 ======
<title="Just like me"> Just like me, I am not to blame I’m just a man and I’ve just had enough I m gone I’m been on death row for too long I took the train to Atlanta to make her smile and she took the bus to Atlanta to make her ha